In [5]:
# PLOT pCO2 with Full SOCAT CC minus line, Full SOCAT CC including line, 
# Full SOCAT CC minus line including Are's line

import numpy as np
import iris
import iris.coord_categorisation
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'xx-large',
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
pylab.rcParams.update(params)
#matplotlib.rc('text', usetex = True)

path = '/Data/Scratch/science/bradshaw-tracks/noresm/sampling/'
LINE = 'Denmark-Greenland'
PLOT_START_INDEX = 265
PLOT_END_INDEX = 372
# locations = ['Finland_Germany', 'Europe_FrenchGuyana']
#locations = ['Denmark_Greenland', 'Finland_Germany', 'France_Brazil', 'Germany_Canada',
#             'Iceland_USA', 'Japan_Australia', 'NorthSea', 'Europe_FrenchGuyana',
#            'USA_Australia', 'USA_Japan', 'All_PseudoSOCATv5']
#locations = ['Germany_Canada']
    
abs_cmap = plt.get_cmap('jet')
#abs_cmap = truncate_colormap(abs_cmap, 0.1, 0.9)
diff_cmap = plt.get_cmap('seismic')

#for location in locations:
#    print location
fig = plt.figure(figsize=(25, 14))
full_socat_fname = path + 'SOCAT_full/without_ship/fco2.nc'
pseudo_fname = path + LINE + '/with_are_ship/fco2.nc'
socat_fname = path + LINE + '/without_ship/fco2.nc'
    
pseudo_cube = iris.load_cube(pseudo_fname, 'fco2')
socat_cube = iris.load_cube(socat_fname, 'fco2')
full_socat_cube = iris.load_cube(full_socat_fname, 'fco2')

pseudo_cube_ex = pseudo_cube.extract(iris.Constraint(time=lambda time: time >= PLOT_START_INDEX))
socat_cube_ex = socat_cube.extract(iris.Constraint(time=lambda time: time >= PLOT_START_INDEX))
full_socat_cube_ex = full_socat_cube.extract(iris.Constraint(time=lambda time: time >= PLOT_START_INDEX))

all_pseudo_cube = pseudo_cube_ex.collapsed(['time'], iris.analysis.MEAN)
all_socat_cube = socat_cube_ex.collapsed(['time'], iris.analysis.MEAN)
all_full_socat_cube = full_socat_cube_ex.collapsed(['time'], iris.analysis.MEAN)

all_diff_cube1 = all_full_socat_cube - all_socat_cube
all_diff_cube2 = all_pseudo_cube - all_socat_cube
diff_diff_cube = all_diff_cube1 - all_diff_cube2

pCO2_socat_real = all_full_socat_cube.data
pCO2_socat_without = all_socat_cube.data
pCO2_socat_pseudo = all_pseudo_cube.data
pCO2_diff_socat_with_without_real = all_diff_cube1.data
pCO2_diff_socat_with_without_pseudo = all_diff_cube2.data
pCO2_diff_diff = diff_diff_cube.data
socat_lats = all_socat_cube.coord('latitude').points
socat_lons = all_socat_cube.coord('longitude').points

ax_socat_real = plt.subplot2grid((3, 6), (0, 0), colspan=2, projection=ccrs.PlateCarree())
ax_socat_without = plt.subplot2grid((3, 6), (0, 2), colspan=2, projection=ccrs.PlateCarree())
ax_socat_pseudo = plt.subplot2grid((3, 6), (0, 4), colspan=2, projection=ccrs.PlateCarree())
ax_diff_socat_real_without = plt.subplot2grid((3, 6), (1, 1), colspan=2, projection=ccrs.PlateCarree())
ax_diff_socat_real_pseudo = plt.subplot2grid((3, 6), (1, 3), colspan=2, projection=ccrs.PlateCarree())
ax_diff_diff = plt.subplot2grid((3, 6), (2, 2), colspan=2, projection=ccrs.PlateCarree())
   
p_socat_real = ax_socat_real.pcolormesh(socat_lons, socat_lats, pCO2_socat_real, cmap=abs_cmap, vmin=200, vmax=400)
p_socat_without = ax_socat_without.pcolormesh(socat_lons, socat_lats, pCO2_socat_without, cmap=abs_cmap, vmin=200, vmax=400)
p_socat_pseudo = ax_socat_pseudo.pcolormesh(socat_lons, socat_lats, pCO2_socat_pseudo, cmap=abs_cmap, vmin=200, vmax=400)
cbar_ax1 = plt.gcf().add_axes([0.91, 0.66, 0.015, 0.25])
cbar_1 = plt.colorbar(p_socat_pseudo, cax=cbar_ax1, extend = 'both')
cbar_1.set_label('pCO2 ($\\mu$atm)', rotation=90)

p_diff_socat_real_without = ax_diff_socat_real_without.pcolormesh(socat_lons, socat_lats, pCO2_diff_socat_with_without_real, cmap=diff_cmap, vmin=-50, vmax=50)
p_diff_socat_real_without = ax_diff_socat_real_pseudo.pcolormesh(socat_lons, socat_lats, pCO2_diff_socat_with_without_pseudo, cmap=diff_cmap, vmin=-50, vmax=50)
cbar_ax2 = plt.gcf().add_axes([0.76, 0.34, 0.015, 0.25])
cbar_2 = plt.colorbar(p_diff_socat_real_without, cax=cbar_ax2, extend = 'both')
cbar_2.set_label('pCO2 anomaly ($\\mu$atm)', rotation=90)

p_diff_diff = ax_diff_diff.pcolormesh(socat_lons, socat_lats, pCO2_diff_diff, cmap=diff_cmap, vmin=-10, vmax=10)
cbar_ax3 = plt.gcf().add_axes([0.61, 0.03, 0.015, 0.25])
cbar_3 = plt.colorbar(p_diff_diff, cax=cbar_ax3, extend = 'both')
cbar_3.set_label('pCO2 anomaly ($\\mu$atm)', rotation=90)

    
titles = ['1: Full SOCATv5 CC (inc SOCATv5 line)',
          '2: Full SOCATv5 CC (ex SOCATv5 line)',
          '3: Full SOCATv5 CC (inc PseudoSOCATv5 line)',
          '4: 1 - 2',
          '5: 3 - 2',
          '6: 4 - 5'
         ]
for ax, title in zip([ax_socat_real, ax_socat_without, ax_socat_pseudo, ax_diff_socat_real_without, ax_diff_socat_real_pseudo, ax_diff_diff], titles):
    ax.add_feature(cartopy.feature.LAND)
    ax.add_feature(cartopy.feature.COASTLINE)
    plt.gca().set_title = title
    ax.title.set_text(title)
    
    

map_plot_dic = {'bottom':0.01, 'top':0.93, 'left':0.01, 'right':0.89, 'wspace':0.2, 'hspace':0.1}
plt.subplots_adjust(**map_plot_dic)

fig.suptitle(LINE + ' 2007-2016 average', fontsize=30, fontweight='bold')

fname = path + LINE + '/' + LINE + '_pco2_validation_map.png'
fig.savefig(fname, dpi=300)
print fname
plt.close()






[  1.25   3.75   6.25   8.75  11.25  13.75  16.25  18.75  21.25  23.75
  26.25  28.75  31.25  33.75  36.25  38.75  41.25  43.75  46.25  48.75
  51.25  53.75  56.25  58.75  61.25  63.75  66.25  68.75  71.25  73.75
  76.25  78.75  81.25  83.75  86.25  88.75  91.25  93.75  96.25  98.75
 101.25 103.75 106.25 108.75 111.25 113.75 116.25 118.75 121.25 123.75
 126.25 128.75 131.25 133.75 136.25 138.75 141.25 143.75 146.25 148.75
 151.25 153.75 156.25 158.75 161.25 163.75 166.25 168.75 171.25 173.75
 176.25 178.75 181.25 183.75 186.25 188.75 191.25 193.75 196.25 198.75
 201.25 203.75 206.25 208.75 211.25 213.75 216.25 218.75 221.25 223.75
 226.25 228.75 231.25 233.75 236.25 238.75 241.25 243.75 246.25 248.75
 251.25 253.75 256.25 258.75 261.25 263.75 266.25 268.75 271.25 273.75
 276.25 278.75 281.25 283.75 286.25 288.75 291.25 293.75 296.25 298.75
 301.25 303.75 306.25 308.75 311.25 313.75 316.25 318.75 321.25 323.75
 326.25 328.75 331.25 333.75 336.25 338.75 341.25 343.75 346.25 348.75
 351.2

ValueError: This operation cannot be performed as there are differing coordinates (latitude, longitude) remaining which cannot be ignored.

<Figure size 1800x1008 with 0 Axes>